In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input
from keras.models import Model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import optimizers  
from keras.callbacks import EarlyStopping 
from keras.callbacks import ModelCheckpoint

In [2]:
# MNIST 데이터셋 불러오기
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [4]:
# 60000, 가로, 세로, 필터
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


In [5]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [6]:
inputs = Input(shape=(28, 28, 1))

h1 = Conv2D(32, kernel_size = (5, 5), activation='relu', padding="same")(inputs)
h1 = MaxPooling2D(pool_size=(2, 2))(h1)

h2 = Conv2D(64, kernel_size = (5, 5), activation='relu', padding="same")(h1)
h2 = MaxPooling2D(pool_size=(2, 2))(h2)

h3 = Flatten()(h2)
h3 = Dense(1000)(h3)

output = Dense(10, activation='softmax')(h3)
outputs = output

model = Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0     

In [7]:
# 모델 컴파일 하기
model_checkpoint = ModelCheckpoint(filepath =  "./model.h5", monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3)

# 모델 학습과정 설정하기
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.SGD(lr = 0.001), metrics=['accuracy'])

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [8]:
# fit() 메서드로 모델 훈련 시키기
hist = model.fit(
    x_train, y_train,
    epochs = 50, batch_size = 128,
    validation_data=(x_test, y_test),
    callbacks = [model_checkpoint, early_stopping]
    )

Epoch 1/50
469/469 [==============================] - ETA: 0s - loss: 2.0490 - accuracy: 0.5849
Epoch 1: val_loss improved from inf to 0.31493, saving model to ./model.h5
469/469 [==============================] - 13s 7ms/step - loss: 2.0490 - accuracy: 0.5849 - val_loss: 0.3149 - val_accuracy: 0.9045
Epoch 2/50
468/469 [============================>.] - ETA: 0s - loss: 0.2383 - accuracy: 0.9262
Epoch 2: val_loss improved from 0.31493 to 0.17095, saving model to ./model.h5
469/469 [==============================] - 3s 7ms/step - loss: 0.2382 - accuracy: 0.9262 - val_loss: 0.1710 - val_accuracy: 0.9465
Epoch 3/50
463/469 [============================>.] - ETA: 0s - loss: 0.1546 - accuracy: 0.9517
Epoch 3: val_loss improved from 0.17095 to 0.12488, saving model to ./model.h5
469/469 [==============================] - 3s 7ms/step - loss: 0.1545 - accuracy: 0.9518 - val_loss: 0.1249 - val_accuracy: 0.9598
Epoch 4/50
463/469 [============================>.] - ETA: 0s - loss: 0.1196 - accura

In [9]:
# 테스트 데이터로 정확도 측정하기
test_loss, test_acc = model.evaluate(x_test, y_test)
print('test_acc: ', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0551 - accuracy: 0.9806
test_acc:  0.9805999994277954


In [10]:
predit = model.predict(x_test)
#for i in range(0,len(predit)) :
#    print("예측값 : ", predit[i], "\t실제값 : ", test_Y[i])
print(predit[0:5])
print(y_test[0 :5])

313/313 [==============================] - 1s 2ms/step
[[1.70172854e-09 9.31075661e-09 1.06696652e-08 6.95315839e-06
  2.65839839e-09 1.12051977e-08 4.96019892e-13 9.99992847e-01
  1.37294185e-08 5.00952773e-08]
 [2.15764517e-09 1.91737004e-09 1.00000000e+00 2.17247020e-09
  1.13277355e-13 8.67953035e-14 5.00727859e-09 4.13664545e-12
  1.67806957e-09 2.82309901e-13]
 [1.02777847e-06 9.99679446e-01 8.92219020e-07 7.56224892e-07
  2.37317072e-04 5.56570853e-07 1.36547669e-05 2.16397148e-05
  4.45633923e-05 1.61245438e-07]
 [9.99857306e-01 1.65487277e-10 2.06481400e-05 8.45375103e-10
  2.42857651e-10 5.34738263e-08 1.21930912e-04 7.72435094e-09
  8.84084557e-08 6.22397511e-09]
 [5.30989386e-09 1.69289684e-08 1.34398548e-10 1.31114412e-12
  9.99997854e-01 1.95861816e-11 6.57341959e-10 2.06255254e-06
  8.40019894e-08 7.29973664e-08]]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0